In [1]:
!pip install jdatetime -q

In [81]:
from datetime import datetime
import warnings
from jdatetime import datetime as jdatetime
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

file_path = '/content/Train_drug demand_completed.csv'
Weather_path = '/content/WeatherFrom2019To2023.csv'
weights_path = '/content/Model_weights.csv'
df=pd.read_csv(file_path)
weather_df=pd.read_csv(Weather_path)
weight_df=pd.read_csv(weights_path)
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])
WEATHER_API_KEY = ""
SCALER = MinMaxScaler()
warnings.filterwarnings("ignore", category=UserWarning)


In [82]:
def Preprocessing_DF(df):
  df['تاریخ مصرف'] = pd.to_datetime(df['تاریخ مصرف'], format='%m/%d/%Y')
  columns_to_drop = ['تخفیفات', 'باز یا بسته بودن داروخانه', 'تعطیلی مدارس', 'تعطیلی هفتگی','کد مشتریان']
  df = df.drop(columns=columns_to_drop, axis=1)
  print("Columns after dropping:", df.columns)
  #simple preprocessings:
  condition = (df['نام بیمارستان'] == 1) & (df['تعداد مصرف'] > 400)
  df.loc[condition, 'تعداد مصرف'] = 100

  condition = (df['نام بیمارستان'] == 4) & (df['تعداد مصرف'] > 20)
  df.loc[condition, 'تعداد مصرف'] = 20

  condition = (df['نام بیمارستان'] == 5) & (df['تعداد مصرف'] > 70)
  df.loc[condition, 'تعداد مصرف'] = 70

  condition = (df['نام بیمارستان'] == 9) & (df['تعداد مصرف'] > 40)
  df.loc[condition, 'تعداد مصرف'] = 40

  condition = (df['نام بیمارستان'] == 10) & (df['تعداد مصرف'] > 100)
  df.loc[condition, 'تعداد مصرف'] = 100

  condition = (df['نام بیمارستان'] == 12) & (df['تعداد مصرف'] > 25)
  df.loc[condition, 'تعداد مصرف'] = 25

  duplicates = df[df.duplicated(['نام بیمارستان', 'تاریخ مصرف','روز هفته'], keep=False)]

  if duplicates.empty:
      print("No duplicates found.")
  else:
      print("Duplicates found. summing duplicates...")

  summed_duplicates = duplicates.groupby(['نام بیمارستان', 'تاریخ مصرف','روز هفته'])['تعداد مصرف'].sum().reset_index()
  df = pd.concat([df[~df.duplicated(['نام بیمارستان', 'تاریخ مصرف','روز هفته'], keep=False)], summed_duplicates], ignore_index=True)
  duplicates = df[df.duplicated(['نام بیمارستان', 'تاریخ مصرف'], keep=False)]

  if duplicates.empty:
      print("No duplicates found.")
  else:
      print("Duplicates found again:")
      print(duplicates)
  date_range = pd.date_range(start='2019-01-01', end='2022-12-31', freq='D')
  multi_index = pd.MultiIndex.from_product([range(1, 13), date_range], names=['نام بیمارستان', 'تاریخ مصرف'])
  df = df.set_index(['نام بیمارستان', 'تاریخ مصرف']).reindex(multi_index, fill_value=0).reset_index()
  df = df.sort_values(by=['تاریخ مصرف', 'نام بیمارستان'])
  return df

In [83]:
df = Preprocessing_DF(df)

Columns after dropping: Index(['نام بیمارستان', 'روز هفته', 'تاریخ مصرف', 'تعداد مصرف'], dtype='object')
Duplicates found. summing duplicates...
No duplicates found.


In [84]:
df['تاریخ مصرف'] = pd.to_datetime(df['تاریخ مصرف'])

start_date = datetime(2019, 1, 1)
end_date = datetime(2022, 12, 31)
new_rows = pd.DataFrame(columns=df.columns)

current_date = start_date
while current_date <= end_date:
    date_filter = (df['تاریخ مصرف'] == current_date)
    total_consumption = df[date_filter]['تعداد مصرف'].sum()
    new_row = pd.DataFrame([[13, current_date,None, total_consumption]], columns=df.columns)
    new_rows = pd.concat([new_rows, new_row], ignore_index=True)

    current_date += timedelta(days=1)

df = pd.concat([df, new_rows], ignore_index=True)

In [85]:
import requests

# Add some extra features:
from jdatetime import date
def check_school_holidays(row):
    persian_date = date.fromgregorian(day=row['تاریخ مصرف'].day, month=row['تاریخ مصرف'].month, year=row['تاریخ مصرف'].year)
    if (
        (4 <= persian_date.month <= 6) or
        (persian_date.month == 1 and 1 <= persian_date.day <= 14) or
        (row['روز هفته'] in [5, 6])
    ):
        return 1
    else:
        return 0
def check_corona_status(row):
    persian_date = date.fromgregorian(day=row['تاریخ مصرف'].day, month=row['تاریخ مصرف'].month, year=row['تاریخ مصرف'].year)
    if (date(1399, 2, 1) <= persian_date <= date(1399, 3, 31)) or (date(1400, 2, 1) <= persian_date <= date(1400, 5, 31)):
        return 2
    elif (date(1398, 12, 1) <= persian_date <= date(1400, 10, 1)):
        return 1
    else:
        return 0
def check_winter_season(row):
    persian_date = date.fromgregorian(day=row['تاریخ مصرف'].day, month=row['تاریخ مصرف'].month, year=row['تاریخ مصرف'].year)

    if persian_date.month in [4, 5]:
        return 0
    elif persian_date.month in [3, 6]:
        return 1
    elif persian_date.month in [2, 7, 8]:
        return 2
    elif persian_date.month in [1, 12, 9]:
        return 3
    elif persian_date.month in [10, 11]:
        return 4
    else:
        return -1  # Handle any unexpected cases
def merge_weather_data(row):
    date_to_match = row['تاریخ مصرف'].date()
    matching_row = weather_df[weather_df['datetime'].dt.date == date_to_match]

    if not matching_row.empty:
        return ','.join(map(str, matching_row.iloc[0][weather_df.columns].values))
    else:
        print(f"We don't have date = {date_to_match.strftime('%Y-%m-%d')} in weather_df.")
        global WEATHER_API_KEY
        url = "https://api.weatherbit.io/v2.0/history/daily"
        start_date = date_to_match.strftime('%Y-%m-%d')
        end_date = (date_to_match + timedelta(days=1)).strftime('%Y-%m-%d')
        params = {
            "lat": 34.7983,
            "lon": 48.5148,
            "start_date": start_date,
            "end_date": end_date,
            "key": WEATHER_API_KEY
        }
        response = requests.get(url, params=params)
        data = response.json()

        data_list = data.get('data', [])
        if not data_list:
            print(f"Error in downloading weather info in date = {date_to_match.strftime('%Y-%m-%d')}: {response}")
            raise ValueError(response)

        cache_df = pd.json_normalize(data_list)
        values_str = ','.join(map(str, cache_df.iloc[0][weather_df.columns].values))
        print(f"Weather data for {date_to_match.strftime('%Y-%m-%d')} updated with new information.")
        return values_str

def day_of_year_feature(row):
    persian_date = date.fromgregorian(day=row['تاریخ مصرف'].day, month=row['تاریخ مصرف'].month, year=row['تاریخ مصرف'].year)
    return (persian_date.toordinal() - date(persian_date.year, 1, 1).toordinal() + 1)

def day_of_Month_feature(row):
    persian_date = date.fromgregorian(day=row['تاریخ مصرف'].day, month=row['تاریخ مصرف'].month, year=row['تاریخ مصرف'].year)
    return (persian_date.day)


In [86]:
df['روز هفته'] = (df['تاریخ مصرف'].dt.dayofweek + 2) % 7
df['تعطیلات مدارس'] = df.apply(check_school_holidays, axis=1)
df['کرونا'] = df.apply(check_corona_status, axis=1)
df['فصل سرما'] = df.apply(check_winter_season, axis=1)
df['روز سال'] = df.apply(day_of_year_feature, axis=1)
df['روز ماه'] = df.apply(day_of_Month_feature, axis=1)

In [87]:
df[['datetime','max_temp', 'min_temp', 'temp', 'rh', 'precip', 'wind_spd', 'solar_rad', 'snow']] = df.apply(merge_weather_data, axis=1).str.split(',', expand=True)
df = df.drop('datetime', axis=1)

In [88]:
columns_to_scale = ['روز هفته','تعطیلات مدارس','کرونا','فصل سرما','روز سال','روز ماه','max_temp', 'min_temp', 'temp',
                    'rh', 'precip', 'wind_spd', 'solar_rad', 'snow']

data_to_scale = df[columns_to_scale]

scaled_data = SCALER.fit_transform(data_to_scale)

df[columns_to_scale] = scaled_data

In [104]:
# Models are gonna use main_df as thier storage to generate data needed as input for labels.
main_df = df.copy()

In [105]:
main_df

,نام بیمارستان,تاریخ مصرف,روز هفته,تعداد مصرف,تعطیلات مدارس,کرونا,فصل سرما,روز سال,روز ماه,max_temp,min_temp,temp,rh,precip,wind_spd,solar_rad,snow
0,1,2019-01-01,0.500000,61.0,0.0,0.0,1.0,0.783562,0.333333,0.354167,0.333333,0.332558,0.623596,0.0,0.211268,0.250712,0.0
1,2,2019-01-01,0.500000,72.0,0.0,0.0,1.0,0.783562,0.333333,0.354167,0.333333,0.332558,0.623596,0.0,0.211268,0.250712,0.0
2,3,2019-01-01,0.500000,156.0,0.0,0.0,1.0,0.783562,0.333333,0.354167,0.333333,0.332558,0.623596,0.0,0.211268,0.250712,0.0
3,4,2019-01-01,0.500000,0.0,0.0,0.0,1.0,0.783562,0.333333,0.354167,0.333333,0.332558,0.623596,0.0,0.211268,0.250712,0.0
4,5,2019-01-01,0.500000,31.0,0.0,0.0,1.0,0.783562,0.333333,0.354167,0.333333,0.332558,0.623596,0.0,0.211268,0.250712,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18988,13,2022-12-27,0.500000,276.0,0.0,0.0,1.0,0.769863,0.166667,0.208333,0.241379,0.232558,0.866292,0.0,0.140845,0.307692,0.0
18989,13,2022-12-28,0.666667,313.0,0.0,0.0,1.0,0.772603,0.200000,0.229167,0.287356,0.248837,0.792135,0.0,0.169014,0.336182,0.0
18990,13,2022-12-29,0.833333,295.0,1.0,0.0,1.0,0.775342,0.233333,0.270833,0.218391,0.218605,0.758427,0.0,0.140845,0.336182,0.0
18991,13,2022-12-30,1.000000,309.0,1.0,0.0,1.0,0.778082,0.266667,0.291667,0.241379,0.241860,0.768539,0.0,0.140845,0.339031,0.0


In [94]:
def insertIntoMain_DF(Date,HospitalName,recursivePrecition):
  # Used for inserting new predictions into main_df.
  global main_df
  new_row = {'نام بیمارستان': HospitalName, 'تاریخ مصرف': Date, 'تعداد مصرف':recursivePrecition}
  main_df.loc[len(main_df)] = new_row
  main_df.loc[len(main_df) - 1,'روز هفته'] = (main_df.loc[len(main_df) - 1, 'تاریخ مصرف'].dayofweek + 2) % 7
  main_df.loc[len(main_df) - 1, 'تعطیلات مدارس'] = check_school_holidays(main_df.loc[len(main_df) - 1])
  main_df.loc[len(main_df) - 1, 'کرونا'] = check_corona_status(main_df.loc[len(main_df) - 1])
  main_df.loc[len(main_df) - 1, 'فصل سرما'] = check_winter_season(main_df.loc[len(main_df) - 1])
  main_df.loc[len(main_df) - 1, 'روز سال'] = day_of_year_feature(main_df.loc[len(main_df) - 1])
  main_df.loc[len(main_df) - 1, 'روز ماه'] = day_of_Month_feature(main_df.loc[len(main_df) - 1])

  try:
    main_df.loc[len(main_df) - 1,['datetime', 'max_temp', 'min_temp', 'temp', 'rh', 'precip', 'wind_spd', 'solar_rad', 'snow']] = merge_weather_data(main_df.loc[len(main_df) - 1]).split(',')
  except:
    main_df.loc[len(main_df) - 1,['datetime', 'max_temp', 'min_temp', 'temp', 'rh', 'precip', 'wind_spd', 'solar_rad', 'snow']] = merge_weather_data(main_df.loc[len(main_df) - 1]).str.split(',', expand=True)


  columns_to_scale = ['روز هفته','تعطیلات مدارس','کرونا','فصل سرما','روز سال','روز ماه','max_temp', 'min_temp', 'temp',
                    'rh', 'precip', 'wind_spd', 'solar_rad', 'snow']
  main_df = main_df.drop('datetime', axis=1)
  last_row_index = len(main_df) - 1
  test_data_to_scale = main_df.loc[last_row_index, columns_to_scale].values.reshape(1, -1)
  scaled_test_data = SCALER.transform(test_data_to_scale)
  main_df.loc[last_row_index, columns_to_scale] = scaled_test_data.flatten()



In [106]:
# Make Dataset Ready to use.
goly = pd.read_csv('/content/goly_drug_test_output.csv', encoding='utf-8')
goly['تاریخ مصرف'] = pd.to_datetime(goly['تاریخ مصرف'], format='%m/%d/%Y')
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 10, 30)
date_range = pd.date_range(start=start_date, end=end_date)
hospitals = goly['نام بیمارستان'].unique()
all_combinations = pd.MultiIndex.from_product([hospitals, date_range], names=['نام بیمارستان', 'تاریخ مصرف'])
missing_combinations = all_combinations.difference(goly.set_index(['نام بیمارستان', 'تاریخ مصرف']).index)
missing_combinations_df = pd.DataFrame(index=missing_combinations).reset_index()
missing_combinations_df['تعداد مصرف'] = 0
goly = pd.concat([goly, missing_combinations_df])
goly = goly.sort_values(by=['تاریخ مصرف','نام بیمارستان'])
gigaGoly = goly[goly['تعداد مصرف'] == 0]
for index, row in gigaGoly.iterrows():
    insertIntoMain_DF(row['تاریخ مصرف'], row['نام بیمارستان'], row['تعداد مصرف'])
main_df = main_df.sort_values(by=['تاریخ مصرف','نام بیمارستان'])
new_hospital_data = {'تاریخ مصرف': date_range, 'نام بیمارستان': '13', 'تعداد مصرف': None}
new_hospital_df = pd.DataFrame(new_hospital_data)
goly = pd.concat([goly, new_hospital_df], ignore_index=True)
goly = goly.sort_values(by=['تاریخ مصرف','نام بیمارستان'])

In [17]:
def thisYearDF(labelDate, HospitalName, seqSize=30, dropDate = True, ForModel_1_4 = False):
    # Returns a Sequence of (seqSize) that are close to (labelDate) for (HospitalName).
    if labelDate <= pd.to_datetime('2018-12-31') + relativedelta(days=seqSize):
        print(f"GolyWarning: The provided labelDate for thisYearDF() is not suitable and its receiving 2018 data as NaN. ({labelDate})")
    labelDate = labelDate - relativedelta(days=1)
    custom_df = pd.DataFrame({
        'نام بیمارستان': [HospitalName] * seqSize,
        'روز هفته': np.nan,
        'تاریخ مصرف': pd.date_range(end=labelDate, periods=seqSize),
        'تعداد مصرف': np.nan,
        'تعطیلات مدارس': np.nan,
        'کرونا': np.nan,
        'فصل سرما': np.nan
    })
    custom_df = custom_df.merge(main_df[['نام بیمارستان','روز هفته', 'تاریخ مصرف', 'تعداد مصرف','تعطیلات مدارس','کرونا','فصل سرما']], on=['نام بیمارستان', 'تاریخ مصرف'], how='left')
    custom_df = custom_df.drop('تعداد مصرف_x', axis=1)
    custom_df = custom_df.drop('روز هفته_x', axis=1)
    custom_df = custom_df.drop('تعطیلات مدارس_x', axis=1)
    custom_df = custom_df.drop('کرونا_x', axis=1)
    custom_df = custom_df.drop('فصل سرما_x', axis=1)

    custom_df = custom_df.rename(columns={'تعطیلات مدارس_y': 'تعطیلات مدارس'})
    custom_df = custom_df.rename(columns={'کرونا_y': 'کرونا'})
    custom_df = custom_df.rename(columns={'فصل سرما_y': 'فصل سرما'})
    custom_df = custom_df.rename(columns={'تعداد مصرف_y': 'تعداد مصرف'})
    custom_df = custom_df.rename(columns={'روز هفته_y': 'روز هفته'})

    custom_df.sort_values(by='تاریخ مصرف', inplace=True)
    this_year_sequence = custom_df.tail(seqSize)

    this_year_sequence['روز هفته'] = (this_year_sequence['تاریخ مصرف'].dt.dayofweek + 2) % 7
    this_year_sequence['تعطیلات مدارس'] = this_year_sequence.apply(check_school_holidays, axis=1)
    this_year_sequence['کرونا'] = this_year_sequence.apply(check_corona_status, axis=1)
    this_year_sequence['فصل سرما'] = this_year_sequence.apply(check_winter_season, axis=1)
    this_year_sequence['روز سال'] = this_year_sequence.apply(day_of_year_feature, axis=1)
    this_year_sequence['روز ماه'] = this_year_sequence.apply(day_of_Month_feature, axis=1)

    this_year_sequence[['datetime','max_temp', 'min_temp', 'temp', 'rh', 'precip', 'wind_spd', 'solar_rad', 'snow']] = this_year_sequence.apply(merge_weather_data, axis=1).str.split(',', expand=True)

    #SCALAR:
    columns_to_scale = ['روز هفته','تعطیلات مدارس','کرونا','فصل سرما','روز سال','روز ماه','max_temp', 'min_temp', 'temp',
                    'rh', 'precip', 'wind_spd', 'solar_rad', 'snow']
    test_data_to_scale = this_year_sequence[columns_to_scale]
    scaled_test_data = SCALER.transform(test_data_to_scale)
    this_year_sequence[columns_to_scale] = scaled_test_data

    if dropDate: this_year_sequence = this_year_sequence.drop('تاریخ مصرف', axis=1)
    if dropDate: this_year_sequence = this_year_sequence.drop('نام بیمارستان', axis=1)
    if dropDate:
      desired_order = ['تعداد مصرف','روز هفته','تعطیلات مدارس','کرونا','فصل سرما','روز سال','روز ماه','max_temp', 'min_temp', 'temp',
                    'rh', 'precip', 'wind_spd', 'solar_rad', 'snow']
    else: desired_order = ['نام بیمارستان','تاریخ مصرف','تعداد مصرف','روز هفته','تعطیلات مدارس','کرونا','فصل سرما','روز سال','روز ماه',
                           'max_temp', 'min_temp', 'temp','rh', 'precip', 'wind_spd', 'solar_rad', 'snow']
    #return this_year_sequence
    Final = this_year_sequence[desired_order]

    return Final


In [18]:
def lastYearDF(labelDate, HospitalName, seqSize=30):
    # Same function like thisYearDF, but for last year, without considering NaNs. When we use this function we make sure this data exists.
    last_year_df = main_df[(main_df['نام بیمارستان'] == HospitalName) & (main_df['تاریخ مصرف'] <= labelDate - pd.DateOffset(years=1))]
    last_year_df = last_year_df.sort_values(by='تاریخ مصرف').copy()
    last_year_sequence = last_year_df.tail(seqSize)
    last_year_sequence = last_year_sequence.drop('تاریخ مصرف', axis=1)
    last_year_sequence = last_year_sequence.drop('نام بیمارستان', axis=1)
    if len(last_year_sequence) != seqSize:
        print(f"GolyWarning: LastYear does not exists in main_df for labelDate={labelDate}, Hospital={HospitalName}.")
    desired_order = ['تعداد مصرف','روز هفته','تعطیلات مدارس','کرونا','فصل سرما','روز سال','روز ماه','max_temp', 'min_temp', 'temp',
                    'rh', 'precip', 'wind_spd', 'solar_rad', 'snow']
    Final = last_year_sequence[desired_order]
    return Final

In [19]:
def generateANumber(labelDate, HospitalName, daysAwayFromLabel):
    # Simply find the number of sales on day (labelDate - daysAwayFromLabel)
    hospital_df = main_df[main_df['نام بیمارستان'] == HospitalName]
    last_year_date = labelDate - relativedelta(days=daysAwayFromLabel)
    last_year_df = hospital_df[
        (hospital_df['تاریخ مصرف'].dt.day == last_year_date.day) &
        (hospital_df['تاریخ مصرف'].dt.month == last_year_date.month) &
        (hospital_df['تاریخ مصرف'].dt.year == last_year_date.year)
    ]
    if last_year_df.empty:
        print(f'GolyWarning: returning -1 for date = {last_year_date} from generateANumber')
        return -1

    last_year_label = last_year_df['تعداد مصرف'].sum()
    return last_year_label

In [20]:
def lastYearLabel(labelDate, HospitalName):
    hospital_df = main_df[main_df['نام بیمارستان'] == HospitalName]

    last_year_date = labelDate - relativedelta(years=1)
    last_year_df = hospital_df[
        (hospital_df['تاریخ مصرف'].dt.day == last_year_date.day) &
        (hospital_df['تاریخ مصرف'].dt.month == last_year_date.month) &
        (hospital_df['تاریخ مصرف'].dt.year == last_year_date.year)
    ]
    if last_year_df.empty:
        return -1

    last_year_label = last_year_df['تعداد مصرف'].sum()
    return last_year_label

In [21]:
def lastMonthLabel(labelDate, HospitalName):
    hospital_df = main_df[main_df['نام بیمارستان'] == HospitalName]

    last_year_date = labelDate - relativedelta(months=1)
    last_year_df = hospital_df[
        (hospital_df['تاریخ مصرف'].dt.day == last_year_date.day) &
        (hospital_df['تاریخ مصرف'].dt.month == last_year_date.month) &
        (hospital_df['تاریخ مصرف'].dt.year == last_year_date.year)
    ]
    if last_year_df.empty:
        return -1

    last_month_label = last_year_df['تعداد مصرف'].sum()
    return last_month_label

In [22]:
def lastWeekLabel(labelDate, HospitalName):
    hospital_df = main_df[main_df['نام بیمارستان'] == HospitalName]

    last_year_date = labelDate - relativedelta(days=7)
    last_year_df = hospital_df[
        (hospital_df['تاریخ مصرف'].dt.day == last_year_date.day) &
        (hospital_df['تاریخ مصرف'].dt.month == last_year_date.month) &
        (hospital_df['تاریخ مصرف'].dt.year == last_year_date.year)
    ]
    if last_year_df.empty:
        return -1

    last_week_label = last_year_df['تعداد مصرف'].sum()
    return last_week_label

In [23]:
def generateTrainDataForADate(labelDate,HospitalName,seqSize = 30):
  # Gererates a single row for train data. used in (generateDataset) function.
  First_sequence = thisYearDF(labelDate,HospitalName,seqSize)
  Sec_Sequence = lastYearDF(labelDate,HospitalName,seqSize)
  last_year_label = generateANumber(labelDate,HospitalName,365)
  last_month_label = generateANumber(labelDate,HospitalName,30)
  last_week_label = generateANumber(labelDate,HospitalName,7)
  return First_sequence, Sec_Sequence, last_year_label, last_month_label, last_week_label

In [24]:
def generateTrainDataForADate_WithoutDrugFeature(labelDate,HospitalName,seqSize = 30,delta = 0, ForModel_1_4=False):
  First_sequence = thisYearDF(labelDate - timedelta(days=delta) + timedelta(days=1),HospitalName,seqSize,ForModel_1_4=ForModel_1_4)
  First_sequence = First_sequence.drop('تعداد مصرف', axis=1)
  return First_sequence


In [25]:
def generateDataset(firstLabelDate, lastLabelDate, hospitalName, seqSize=30):
    # Generates Train Dataset using 2 sequence of time-series data. will be used for Model_1. each row of generated dataset is a train-
    # -data for a label ( that is between (firstLabelDate) and (lastLabelDate) ).
    current_date = firstLabelDate
    all_input_1_np = []
    all_input_2_np = []
    all_LY_L = []
    all_LM_L = []
    all_LW_L = []


    while current_date <= lastLabelDate:
        LY_DF, TY_DF, LY_L, LM_L, LW_L = generateTrainDataForADate(current_date, hospitalName, seqSize)
        input_1_np = LY_DF.values.reshape(1, seqSize, 15)
        input_2_np = TY_DF.values.reshape(1, seqSize, 15)
        all_input_1_np.append(input_1_np)
        all_input_2_np.append(input_2_np)
        all_LY_L.append(LY_L)
        all_LM_L.append(LM_L)
        all_LW_L.append(LW_L)
        current_date += timedelta(days=1)

    merged_input_1_np = np.concatenate(all_input_1_np, axis=0)
    merged_input_2_np = np.concatenate(all_input_2_np, axis=0)
    merged_LY_L = np.array(all_LY_L)
    merged_LM_L = np.array(all_LM_L)
    merged_LW_L = np.array(all_LW_L)

    return merged_input_1_np, merged_input_2_np, merged_LY_L, merged_LM_L, merged_LW_L


In [26]:
def generateDataset_WithoutDrugFeature(firstLabelDate, lastLabelDate, hospitalName, seqSize=30 , delta = 0):
    # Generates Train Dataset using 1 sequence of time-series data without DrugUsage Feature. will be used for Model_4.
    current_date = firstLabelDate
    all_input_1_np = []

    while current_date <= lastLabelDate:
        a = generateTrainDataForADate_WithoutDrugFeature(current_date, hospitalName, seqSize, delta, ForModel_1_4=True)
        input_1_np = a.values.reshape(1, seqSize, 14)
        all_input_1_np.append(input_1_np)
        current_date += timedelta(days=1)

    merged_input_1_np = np.concatenate(all_input_1_np, axis=0)


    nan = np.count_nonzero(np.isnan(merged_input_1_np))
    if  nan != 0:
      print(f'GolyWarning: Found {nan} NaN numbers in merged_input_1_np')

    return merged_input_1_np


In [110]:
from tensorflow.keras.models import load_model
Path_To_Models = '/content/drive/MyDrive/Hamedan Models/'
#Load Models:
Model_list = []
for i in range (1,14):
  print(f'Loading Models for Hospital = {i}')
  Model_1 = load_model(Path_To_Models+f'Model 1/M1_seq30_Hospital{i}.h5')
  Model_2 = load_model(Path_To_Models+f'Model 1/M1_seq30_Hospital{i}_LastEpoch.h5')
  Model_3 = load_model(Path_To_Models+f'Model 4/M4_seq28_Hospital{i}.h5')
  Model_4 = load_model(Path_To_Models+f'Model 4/M4_seq28_Hospital{i}_LastEpoch.h5')
  Model_list.append([Model_1,Model_2,Model_3,Model_4])

In [158]:
def Weighted_predict(model_list, labelDate, HospitalName, debug_showPrints = False):
  print(f"Weighted_predict for:{labelDate}-H:{HospitalName}")
  # STEP 1:
  #inputs for seq28 Model_1
  a = thisYearDF(labelDate,HospitalName,30,False)
  b = lastYearDF(labelDate,HospitalName,30)
  b = b.astype(np.float64)
  c = lastYearLabel(labelDate,HospitalName)
  d = lastMonthLabel(labelDate,HospitalName)
  e = lastWeekLabel(labelDate,HospitalName)
  #inputs for seq28 Model_4
  f = thisYearDF(labelDate + timedelta(days=1),HospitalName,28,ForModel_1_4=True)
  f = f.drop('تعداد مصرف', axis=1)
  # STEP 2: inputs of Model_1 might have NaN in their input dataset (مقدار مصرف). lets fix it recuresively:
  a = a.sort_values(by='تاریخ مصرف')
  for index, row in a.iterrows():
    if pd.isna(row['تعداد مصرف']):
        recursivePrecition = Weighted_predict(model_list, row['تاریخ مصرف'], row['نام بیمارستان'])
        recursivePrecition = round(recursivePrecition)
        insertIntoMain_DF(row['تاریخ مصرف'],row['نام بیمارستان'],recursivePrecition)
        a.at[index, 'تعداد مصرف'] = recursivePrecition
  if c==-1: c=Weighted_predict(model_list,(labelDate - relativedelta(years=1)), HospitalName)
  if d==-1: d=Weighted_predict(model_list,(labelDate - relativedelta(months=1)), HospitalName)
  if e==-1: e=Weighted_predict(model_list,(labelDate - relativedelta(days=7)) , HospitalName)

  # STEP 3: Predict:
  desired_order = ['تعداد مصرف','روز هفته','تعطیلات مدارس','کرونا','فصل سرما','روز سال','روز ماه','max_temp', 'min_temp', 'temp',
                    'rh', 'precip', 'wind_spd', 'solar_rad', 'snow']
  a = a.drop(['تاریخ مصرف', 'نام بیمارستان'], axis=1)[desired_order]
  a_reshaped = a.values.reshape((1, 30, 15))
  b_reshaped = b.values.reshape((1, 30, 15))
  c_reshaped = np.array(c).reshape((1, 1))
  d_reshaped = np.array(d).reshape((1, 1))
  e_reshaped = np.array(e).reshape((1, 1))
  f_reshaped = f.values.reshape((1, 28, 14))
  outputs = []
  for i in range (0,2):
    try:
      outputs.append(model_list[i].predict([a_reshaped, b_reshaped, c_reshaped, d_reshaped, e_reshaped],verbose=0))
    except:
      raise ValueError('Something is wrong in Weighted_predict. Check if you passed models in order of [Model_1,Model_1_LastEpoch,Model_4,Model_4_lastEpoch]')

  for i in range (2,4):
    try:
      outputs.append(model_list[i].predict([f_reshaped],verbose=0))
    except:
      raise ValueError('Something is wrong in Weighted_predict. Check if you passed models in order of [Model_1,Model_1_LastEpoch,Model_4,Model_4_lastEpoch]')



  W1 = weight_df.loc[weight_df['Hospital'] == HospitalName, 'Model_1'].values[0]
  W2 = weight_df.loc[weight_df['Hospital'] == HospitalName, 'Model_1_lastEpoch'].values[0]
  W3 = weight_df.loc[weight_df['Hospital'] == HospitalName, 'Model_4'].values[0]
  W4 = weight_df.loc[weight_df['Hospital'] == HospitalName, 'Model_4_lastEpoch'].values[0]

  Final_predict = (outputs[0][0][0] * W1) + (outputs[1][0][0] * W2) + (outputs[2][0][0] * W3) + (outputs[3][0][0] * W4)

  # STEP 4: insert in main_df if it does not exists:
  if not ((main_df['تاریخ مصرف'] == labelDate) & (main_df['نام بیمارستان'] == HospitalName)).any():
    print(f'inserting:{labelDate}', end=" - ")
    insertIntoMain_DF(labelDate,HospitalName,round(Final_predict))

  # STEP 5: return Predicted Value:
  return round(Final_predict)


In [164]:
for index, row in goly.iterrows():
    if pd.isna(row['تعداد مصرف']):
        HospitalName = int(row['نام بیمارستان'])
        goly.at[index, 'تعداد مصرف'] = Weighted_predict(Model_list[HospitalName-1],row['تاریخ مصرف'], HospitalName)


Weighted_predict for:2023-01-01 00:00:00-H:13
inserting:2023-01-01 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:1
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:2
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:3
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:4
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:5
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:6
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:7
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:8
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:9
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:10
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:11
inserting:2023-01-02 00:00:00 - Weighted_predict for:2023-01-02 00:00:00-H:12
inserting:2023-01-02 00:00:

In [165]:
goly

,نام بیمارستان,تاریخ مصرف,تعداد مصرف
0,1,2023-01-01,37.0
1,2,2023-01-01,39.0
2,3,2023-01-01,118.0
3,4,2023-01-01,1.0
4,5,2023-01-01,21.0
...,...,...,...
3641,9,2023-10-30,19.0
3642,10,2023-10-30,31.0
3643,11,2023-10-30,1.0
3644,12,2023-10-30,0.0


In [167]:
goly.to_csv('Final Output - Daily.csv', encoding='utf-8', index=False)


In [181]:
# STEP 1: Daily drug usage for each hospital
# Final output has hospital 13 too. so we need to reload goly_drug_test_output (which is the first 2 columns of drug_test_output)
df1 = pd.read_csv('/content/goly_drug_test_output.csv')
df2 = pd.read_csv('/content/Final Output - Daily.csv')
df1['تاریخ مصرف'] = pd.to_datetime(df1['تاریخ مصرف'])
df2['تاریخ مصرف'] = pd.to_datetime(df2['تاریخ مصرف'])
df2_no_duplicates = df2.drop_duplicates(subset=['نام بیمارستان', 'تاریخ مصرف'])
merged_df = pd.merge(df1, df2_no_duplicates[['نام بیمارستان', 'تاریخ مصرف', 'تعداد مصرف']], on=['نام بیمارستان', 'تاریخ مصرف'], how='left')
merged_df = merged_df.dropna(subset=['تعداد مصرف'])
merged_df['تعداد مصرف'] = merged_df['تعداد مصرف'].replace(0, 1)
merged_df.to_csv('merged_data.csv', index=False)


In [232]:
# STEP 2: Sum of hospitals
total_usage_by_date = merged_df.groupby('تاریخ مصرف')['تعداد مصرف'].sum().reset_index()
total_usage_by_date.columns = ['تاریخ مصرف', 'مجموع مصرف']
print(total_usage_by_date)

result_df = total_usage_by_date.merge(goly[goly['نام بیمارستان'] == '13'], left_on='تاریخ مصرف', right_on='تاریخ مصرف', how='left')
result_df = result_df.drop('نام بیمارستان',axis = 1)
result_df['میانگین'] = result_df[['مجموع مصرف', 'تعداد مصرف']].mean(axis=1)
result_df['میانگین'] = result_df['میانگین'].astype(int)

# Print the result dataframe
print(result_df)
result_df.to_csv('Daily-All-Hospitals.csv', index=False)


    تاریخ مصرف  مجموع مصرف
0   2023-01-01       290.0
1   2023-01-02       280.0
2   2023-01-03       261.0
3   2023-01-04       273.0
4   2023-01-05       256.0
..         ...         ...
298 2023-10-26       216.0
299 2023-10-27       217.0
300 2023-10-28       216.0
301 2023-10-29       230.0
302 2023-10-30       237.0

[303 rows x 2 columns]
    تاریخ مصرف  مجموع مصرف  تعداد مصرف  میانگین
0   2023-01-01       290.0       296.0      293
1   2023-01-02       280.0       277.0      278
2   2023-01-03       261.0       257.0      259
3   2023-01-04       273.0       263.0      268
4   2023-01-05       256.0       252.0      254
..         ...         ...         ...      ...
298 2023-10-26       216.0       209.0      212
299 2023-10-27       217.0       207.0      212
300 2023-10-28       216.0       211.0      213
301 2023-10-29       230.0       218.0      224
302 2023-10-30       237.0       217.0      227

[303 rows x 4 columns]


In [234]:
# STEP 3: Monthly
result_df['تاریخ مصرف'] = pd.to_datetime(result_df['تاریخ مصرف'])

# Group by month and calculate the sum of 'میانگین' for each month
monthly_usage_df = result_df.groupby(result_df['تاریخ مصرف'].dt.to_period("M")).agg({'میانگین': 'sum'}).reset_index()

# Print the new dataframe
print(monthly_usage_df)
monthly_usage_df.to_csv('Monthly-All-Hospitals.csv', index=False)


  تاریخ مصرف  میانگین
0    2023-01     7081
1    2023-02     6042
2    2023-03     6924
3    2023-04     7502
4    2023-05     7664
5    2023-06     6970
6    2023-07     8326
7    2023-08     8202
8    2023-09     7254
9    2023-10     6421
